In [1]:
import pandas as pd, numpy as np, tsam.timeseriesaggregation as tsam, matplotlib.pyplot as plt

In [2]:
df_load = pd.read_csv("../../data/LoadProfile.csv")
df_load

,Load Profile
0,1355.3
1,1319.9
2,1323.8
3,1294.6
4,1351.7
...,...
8755,1234.2
8756,1283.6
8757,1306.4
8758,1327.6


In [3]:
df0 = pd.DataFrame()
price_dict = {"high"}
price_type = "high"
for price_type in ["low","high"]:
    df = pd.read_csv(f"../../data/price_{price_type}.csv").melt().rename(columns={"value":"price", "variable" : "year"})
    df["scenario_price"] = price_type
    df0 = pd.concat([df,df0],axis = 0)


df = pd.DataFrame((df0.loc[df0["scenario_price"] == "low", "price"] + df0.loc[df0["scenario_price"] == "high", "price"]) / 2)
df["scenario_price"] = "mid"
df["year"] = df0.loc[df0["scenario_price"] == "low", "year"]
df0 = pd.concat([df,df0],axis = 0)
df0["year"] = df0["year"].astype(int)
df = df0[df0['year'].isin([2025,2030,2035,2040,2045,2050])]

df_load_repeated = pd.concat([df_load] * (3 * 6), ignore_index=True)
df['Load Profile'] = df_load_repeated['Load Profile'].values

In [5]:
# Assuming typPeriods is your DataFrame with MultiIndex
def plot_multiindex_data(typPeriods):
    # Get unique values for coloring and x-axis
    first_level_values = typPeriods.index.get_level_values(0).unique()
    second_level_values = typPeriods.index.get_level_values(1).unique()
    columns = typPeriods.columns
    
    # Create subplots - one for each column
    fig, axes = plt.subplots(len(columns), 1, figsize=(12, 4*len(columns)))
    if len(columns) == 1:
        axes = [axes]
    
    # Create consistent color mapping for first index level
    colors = plt.cm.Set3(np.linspace(0, 1, len(first_level_values)))
    color_map = dict(zip(first_level_values, colors))
    
    # Plot each column in a separate subplot
    for ax_idx, column in enumerate(columns):
        ax = axes[ax_idx]
        
        # Plot each first-level index with consistent color
        for first_level in first_level_values:
            color = color_map[first_level]
            
            # Get data for this first-level index
            mask = typPeriods.index.get_level_values(0) == first_level
            subset = typPeriods[mask][column]
            
            # Plot with second level as x-axis
            x_positions = range(len(second_level_values))
            ax.plot(x_positions, subset.values, 
                   marker='o', linewidth=2, markersize=6,
                   color=color, label=f'{first_level}')
        
        ax.set_title(f'{column}')
        ax.set_xlabel('Second Index Level')
        ax.set_ylabel('Values')
        ax.set_xticks(range(len(second_level_values)))
        ax.set_xticklabels(second_level_values)
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()

In [7]:
type_periods = pd.DataFrame

df_all_weeks = pd.DataFrame()
df_all_weights = pd.DataFrame()
df_all_metrics = pd.DataFrame()

for year in df["year"].unique():
    for scen in df["scenario_price"].unique():
        to_aggregate = df[(df["year"] == year) & (df["scenario_price"] == scen)]
        aggregation = tsam.TimeSeriesAggregation(to_aggregate[["Load Profile","price"]],
            resolution = 1,
            noTypicalPeriods = 6,
            hoursPerPeriod = 24*7,
            segmentation = False,
            # noSegments = 6,
            representationMethod = "distributionAndMinMaxRepresentation",
            # distributionPeriodWise = False
            clusterMethod = 'hierarchical',
            addPeakMax = ["Load Profile"]
        )
        typPeriods = aggregation.createTypicalPeriods()
        weights = pd.DataFrame([aggregation.clusterPeriodNoOccur])
        # plot_multiindex_data(typPeriods)
        accuracy = aggregation.accuracyIndicators()
        print(accuracy)
        df_unstacked = typPeriods.reset_index().rename(columns = {"level_0":"typical_week","TimeStep":"hour"})

        info_accuracy = accuracy.copy().reset_index().rename(columns={'index': 'ts_type'})
        info_accuracy["year"] = year
        info_accuracy["scenario_price"] = scen
        df_unstacked["year"] = year
        df_unstacked["scenario_price"] = scen
        weights["year"] = year
        weights["scenario_price"] = scen

        df_all_weeks = pd.concat([df_unstacked,df_all_weeks],axis = 0)
        df_all_metrics = pd.concat([df_all_metrics,info_accuracy])
        df_all_weights = pd.concat([df_all_weights,weights])
    #     break
    # break

df_all_weeks.to_csv("../../data/preprocessed/typical_weeks_all.csv")
df_all_metrics.to_csv("../../data/preprocessed/typical_metrics_all.csv")
df_all_weights.to_csv("../../data/preprocessed/typical_weights_all.csv")

                  RMSE  RMSE_duration       MAE
Load Profile  0.103460       0.002627  0.076685
price         0.064135       0.001473  0.049321
                  RMSE  RMSE_duration       MAE
Load Profile  0.100061       0.002614  0.075438
price         0.066343       0.001545  0.050387
                  RMSE  RMSE_duration       MAE
Load Profile  0.100294       0.002797  0.074231
price         0.066311       0.001975  0.051258
                  RMSE  RMSE_duration       MAE
Load Profile  0.107325       0.002618  0.078715
price         0.061970       0.001731  0.048024
                  RMSE  RMSE_duration       MAE
Load Profile  0.100061       0.002614  0.075438
price         0.066343       0.001545  0.050387
                  RMSE  RMSE_duration       MAE
Load Profile  0.100294       0.002797  0.074231
price         0.066311       0.001975  0.051258
                  RMSE  RMSE_duration       MAE
Load Profile  0.101196       0.002693  0.073944
price         0.067688       0.001849  0